# Adsorption energy calculations using a pre-trained OCP calculator

Now that we've seen how the toy EMT calculator works for the *OCH3 adsorbate ({doc}`relaxations`), let's see how a modern OCP calculator does for the same job. 

To make this easy, we provide an [ASE-compatible calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html) calculator to interface with ASE's functionality. Note that ASE is GPL-licensed!

For this tutorial we download one of the better current best model checkpoint (GemNet-T). This checkpoint has a few important caveats to keep in mind before using it blindly:

* This is trained as a force field to predict both energies and forces for specific configurations
* The energies predicted are referenced to the bare surface and the same linear combination of species in OC20 (see {doc}`relaxations` for details on the reference scheme)
* The energies and forces are not internally consistent, since we've found that models that predict the energy and force separately tend to do better for OC20

## Download pretrained checkpoint

We have released checkpoints of all the models on the leaderboard [here](https://github.com/Open-Catalyst-Project/ocp/blob/master/MODELS.md). These trained models can be used as an ASE calculator for various calculations.

In [ ]:
!wget -q -nc https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt
checkpoint_path = "gemnet_t_direct_h512_all.pt"

## Using the OCP Calculator


In [ ]:
import os

import ase.io
import numpy as np
from ase.build import add_adsorbate, fcc100, molecule
from ase.constraints import FixAtoms
from ase.optimize import LBFGS
from ocpmodels.common.relaxation.ase_utils import OCPCalculator

# Construct a sample structure, similar to the EMT relaxation example!
adslab = fcc100("Cu", size=(3, 3, 3))
adsorbate = molecule("CH3O")
add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)
cons = FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(True)

# Define the calculator
calc = OCPCalculator(checkpoint=checkpoint_path)

# Set up the calculator
adslab.set_calculator(calc)

os.makedirs("data", exist_ok=True)
opt = LBFGS(adslab, trajectory="data/ml_Cu100+CH3O.traj")

opt.run(fmax=0.05, steps=100)

Now, let's visualize the output!

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import animate
from matplotlib import rc

# the `index` argument corresponds to what frame of the trajectory to read in, specifiying ":" reads in the full trajectory.
traj = ase.io.read("data/ml_Cu100+CH3O.traj", index=":")
anim = animate(traj, radii=0.8, rotation=("-75x, 45y, 10z"))

rc("animation", html="jshtml")
anim

Compare the results here with what EMT predicted in {doc}`relaxations`. Note that the adsorbate rotates so that the O is pointing down on the surface and the adsorbate does not associate. This looks much more reasonable!

We can plot the adsorption energy directly since this OCP calculator directly outputs referenced energies. We don't need to repeat the gas-phase and bare slab calculations like we did in {doc}`relaxations`.

In [ ]:
import pandas as pd
import plotly.express as px

energies = [image.get_potential_energy() for image in traj]

df = pd.DataFrame(
    {
        "Relaxation Step": range(len(energies)),
        "GemNet-T Adsorption Energy [eV]": energies,
    }
)
fig = px.line(df, x="Relaxation Step", y="GemNet-T Adsorption Energy [eV]")
fig.show()